# Modul 1.2 Klimasystemet - Toboksmodellen
<img src="Bilder/toboksmodell.png">

Toboksmodellen er en forenklet klimamodell. Den ser på hvordan energien endrer seg i atmosfæren utfra endringer i strålingspådrivene. I figuren over ser dere at man ser på en balanse mellom stråling som går inn og ut på toppen av atmosfæren (TOA), samt de energiovergangene vi har på grensen mellom atmosfæren og det øverste laget i havet, samt at det går noe varme fra havets blandingslag til dyphavet. 

De som ønsker å forstå hvordan man kommer frem til formlene for beregning av temperaturendring som følge av endringer i strålingspådrivet anbefales å lese kompendiet *Climate Sensitivity*. Der vises det hvordan vi kommer frem til likningene under. Disse likningene er kodet og ligger i en Python-fil i mappen deres. Filen heter toboks.py.

$c_{mix}$ og $c_{deep}$ forteller om varmekapasiteten til overflatelaget i havet og i dyphavet. $\Delta T_s$ gir endringer i overflatetemperaturer, mens $\Delta T_o$ endringer i temperaturen i dyphavet. $\lambda$ er tilbakekoblingseffekter, og $\gamma$ varmeopptak i havet.

$$\begin{align}
c_{mix}\dfrac{d\Delta T_s}{dt}&=Q_{adj}-\lambda \Delta T_s -\gamma (\Delta T_s -\Delta T_0) \\
c_{deep}\dfrac{d\Delta T_o}{dt}&=\gamma (\Delta T_s - \Delta T_o)
\end{align}$$

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib notebook inkluder denne linjen om du vil åpne i Jupyter Notebook og vil lage interaktive plott

from toboks import calculate_temp_anomalies # henter funksjonen som beregner temperaturanomalier basert på toboksmodellen

df=pd.read_csv('pmip3.csv',index_col = 0,sep=',',encoding = "utf-8")

## Bestem pådrivene
Noen ganger så ønsker man bare å studere effektene av eksempelvis solens aktivitet. Sett parameterne under til 1 eller 0, alt ettersom du ønsker å ha dem med eller ei. I koden under lages en ny dataramme som inneholder totalt pådriv med disse faktorene.

In [ ]:
# --------------------
# Forcing switches [n=1 off=0]
# --------------------
switch_ghg=1     # Greenhouse gas forcing on=1 off=0
switch_solar=1   # Solar forcing on=1 off=0
switch_volc=1    # Volcanic forcing on=1 off=0
switch_land=1    # Landuse forcing on=1 off=0
switch_aero=1    # Pollution particle forcing on=1 off=0

In [ ]:
total_forcing = switch_ghg*df['drivhusgasser']+switch_solar*df['solinnstråling']+switch_volc*df['vulkanisme']+switch_land*df['arealbruk']+switch_aero*df['aerosoler']
total_forcing.plot(title="Samlet strålingspådriv",ylabel="$W/m^2$")
plt.axhline(y=0, linewidth=0.7, color="black")

## Tilbakekoblingsfaktorer

In [ ]:
# CMIP5 values
lambda_planck=-3.1     # Stefan Boltzmann's 'feedback' [Wm-2K-1] best guesses  [-3.3 to -3.1]
lambda_lapse=-0.6      # Lapse rate feedback [Wm-2K-1] best guesses [-1.2 to -0.5]
lambda_water=1.7       # water vapor feedback [Wm-2K-1] best guesses [1.5 to 2.2]
lambda_cloud=0.50      # Clouds feedback [Wm-2K-1] best guesses [0.1 to 1.2]
lambda_albedo=0.40     # Surface albedo feedback [Wm-2K-1] best guesses [0.1 to 0.4]
lambda_other=0.0       # Other feedbacks [Wm-2K-1]


lambda_sum=sum([lambda_planck,
    lambda_lapse,
    lambda_water,
    lambda_cloud,
    lambda_albedo,
    lambda_other])

In [ ]:
# Deep ocean
# Heat uptake efficency [Wm-2K-1]
# CMIP3 values
gamma=-0.69  # best guesses [-1 to -0.5]

In [ ]:
radiative_forcing = total_forcing.to_numpy()

## Temperaturendringer
Vi ønsker nå å beregne hvordan temperaturen endrer seg som resultat av hvilke strålingspådriv vi har slått på, verdiene vi har satt for tilbakekoblingsmekanismer, og varmeutveksling til havet. 

In [ ]:
Ts, To = calculate_temp_anomalies(radiative_forcing, lambda_sum, gamma) # kaller opp funksjonen som regner ut temperaturendringene

In [ ]:
temp = pd.DataFrame(index=df.index) # vi lager en ny dataramme som har samme indexer som pådrivet
temp['Temp.endring overflate']=Ts
temp['Temp.endring dyphavet']=To
temp.plot(ylabel="$\Delta T$  $[^oC]$", title="Temperaturendringer")
plt.axhline(y=0, linewidth=0.7, color="black")

In [ ]:
temp['Temp.endring dyphavet'].plot()

In [ ]:
temp.plot(subplots=True)

### Se verdier for bestemte år
Velg årstallet du ønsker å vite verdiene for

In [ ]:
temp.loc[1950]

### Nå kan du velge å lage et anomali-plott der du trekker fra verdiene fra et bestemt år

In [ ]:
aarstall=1900 # endre årstallet selv
temp2 = pd.DataFrame(index=df.index)
temp2[f'Endring overflatetemp i forhold til {aarstall}']=temp['Temp.endring overflate']-temp.at[aarstall,'Temp.endring overflate']
temp2.plot(ylabel='$^oC$')
plt.axhline(y=0, linewidth=0.7, color="black")